In [15]:
from jupyter_client import find_connection_file
connection_file = find_connection_file()
print(connection_file)

/home/yonatan/.local/share/jupyter/runtime/kernel-v2-127597fKQxkj6UuQhe.json


In [16]:
import papermill as pm
import pandas as pd
import os
import sys
sys.tracebacklimit = 0
def exception_handler(exception_type, exception, traceback):
    # All your trace are belong to us!
    # your format
    print(f"{exception_type.__name__}, {exception}")

sys.excepthook = exception_handler


In [17]:
dataset_col = "Dataset"
uid_col = "uid"
age_col = "Age"
day_col = "Day"
response_col = "Response"
immage_col = "IMMAGE"
strain_col = 'Strain'

In [18]:
# Read in Data and drop missing values
df = pd.read_csv("../data/all_vaccines.csv")
df.dropna(inplace=True, subset=[immage_col, dataset_col, day_col, response_col])
datasets = df


# Plot distribution of N values
N_vals = df[[dataset_col, uid_col]].groupby(dataset_col, as_index=False)[uid_col].nunique()
N_vals = N_vals.rename(columns={uid_col: "N"})

# Narrow to large datasets only
bNarrow = False
N_vals = N_vals.loc[N_vals["N"] > 70]
if bNarrow:
    datasets = df.loc[df["Dataset"].isin(N_vals["Dataset"])]
dataset_names = datasets["Dataset"].unique()

In [19]:

#Collect day info from papers here
dataset_day_dict = {
        "GSE125921.SDY1529": ['FC', 'D84'],
        "GSE13485.SDY1264": ['D60'],
        "GSE13699.SDY1289": ['D28'],
        "GSE169159": ['FC.D42', 'D42'],
        #"GSE190001": ['D14', 'FC.D14'],
        "GSE41080.SDY212": ['HAI.D28'],
        "GSE45735.SDY224": ['HAI.D21'],
        "GSE47353.SDY80": ['D70.nAb', 'FC.D70.nAb'],
        "GSE48018.SDY1276": ['nAb.D28', 'nAb.FC'],
        "GSE48023.SDY1276": ['nAb.FC', 'nAb.D14'],
        "GSE59635.SDY63": ['HAI.D28'],
        "GSE59654.SDY180": ['FC.HAI', 'HAI.D28'],
        "GSE59654.SDY404": ['FC.HAI', 'HAI.D28'],
        "GSE59654.SDY520": ['FC.HAI', 'HAI.D28'],
        "GSE59743.SDY400": ['FC.HAI', 'HAI.D28'],
        "GSE65834.SDY1328": ['D7', 'FC'],
        "GSE79396.SDY984": ['D28', 'FC.D28'],
        "GSE82152.SDY1294": ['D28', 'FC'],
        "SDY1325": ['FC.D28', 'D28'],
        "SDY296": ['D28.nAb', 'FC.nAb'],
        "SDY67": ['nAb.D28', 'FC.D28.nAb'],
        "SDY89": ['D28']
}
dataset_names = dataset_day_dict.keys()



In [20]:
# Loop through each combination of dataset and strain
for dataset_name in dataset_names:
        dataset = datasets.loc[datasets[dataset_col] == dataset_name] 
        print(dataset_name)
        days = dataset_day_dict[dataset_name]
        for day in days:
                print(day)
                day_mask = dataset[day_col] == day
                name_mask = dataset[dataset_col] == dataset_name
                data = dataset.loc[(name_mask) & (day_mask)].reset_index()
                strains = data[strain_col].unique()
                print(strains)
                for strain_index in range(len(strains)):
                        strain_name = strains[strain_index].replace("/", "_").replace(" ", "_")
                        print(f'exporting {dataset_name}, strain no. {strain_index}: {strain_name}, day: {day}')
                        # Define parameters for dataset and strain
                        parameters = {"dataset_name": dataset_name, "strain_index": strain_index, "day": day}

                        # Execute the notebook with specific parameters
                        output_notebook = f"export/{dataset_name}_{strain_name}_{day}_analysis.ipynb"
                        try:
                                pm.execute_notebook(
                                        input_path="vaccines-4.ipynb",
                                        output_path=output_notebook,
                                        parameters=parameters,
                                        prepare_only=True
                                )
                        except:
                                print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
                        # Export the executed notebook to HTML
                        output_html = f"{dataset_name}_{strain_name}_{day}_analysis.html"
                        os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")

GSE125921.SDY1529
FC
['Yellow Fever']
exporting GSE125921.SDY1529, strain no. 0: Yellow_Fever, day: FC


[NbConvertApp] Converting notebook export/GSE125921.SDY1529_Yellow_Fever_FC_analysis.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 745340 bytes to export/GSE125921.SDY1529_Yellow_Fever_FC_analysis.html


D84
['Yellow Fever']
exporting GSE125921.SDY1529, strain no. 0: Yellow_Fever, day: D84


[NbConvertApp] Converting notebook export/GSE125921.SDY1529_Yellow_Fever_D84_analysis.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 745861 bytes to export/GSE125921.SDY1529_Yellow_Fever_D84_analysis.html


GSE13485.SDY1264
D60
['nAb' 'HLADRpos.CD38pos.CD8pos']
exporting GSE13485.SDY1264, strain no. 0: nAb, day: D60


...

[NbConvertApp] Converting notebook export/SDY67_A_California_7_2009_nAb.D28_analysis.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 819584 bytes to export/SDY67_A_California_7_2009_nAb.D28_analysis.html


FC.D28.nAb
['A/California/7/2009']
exporting SDY67, strain no. 0: A_California_7_2009, day: FC.D28.nAb


[NbConvertApp] Converting notebook export/SDY67_A_California_7_2009_FC.D28.nAb_analysis.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 6 image(s).
[NbConvertApp] Writing 842983 bytes to export/SDY67_A_California_7_2009_FC.D28.nAb_analysis.html


SDY89
D28
['Hepatitis']
exporting SDY89, strain no. 0: Hepatitis, day: D28


[NbConvertApp] Converting notebook export/SDY89_Hepatitis_D28_analysis.ipynb to html
Traceback (most recent call last):
  File "/home/yonatan/miniforge3/envs/vaccines/bin/jupyter-nbconvert", line 10, in <module>
    sys.exit(main())
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/nbconvert/nbconve